# Ezmeral Data Fabric

## Create, delete and update volumes Demonstration

<p align="center">
  <img src="Pictures/edf-volumes-overview.png" width="600" height="400" alt="Data Fabric Snapshots">                                                    
  
</p>

#### Let's review what volumes on a Data Fabric cluster allow for; they allow for management of:

- Security Policies for data in a volume
- Volume administration controls on the volume
- Tiering Policies
- Media placement for data using storage labels
- Business continuity settings using Snapshots
- Disaster recover settings using Mirrors
- Creating schedules for business continuity and disaster recovery
- Audit tracking of data within a volume
- Data at rest encryption (DARE)
- Encryptiong over the wire

#### Within this Ezmeral Data Fabric system your user has been given rights to create/modify volumes.  There are 3 different types of permissions:

1. Is allowing users/groups to make changes to the cluster.
2. Is allowing users/groups to make changes to volumes.
3. Is allowing a user/group to create/modify security polices.

There are many different rights to administrating a cluster, volumes and security policy.  For a full listing please see the [ACL documentation](https://docs.datafabric.hpe.com/62/ReferenceGuide/acl.html?hl=acl).

# Creating Volumes on Ezmeral Data Fabric

## 1. Let's setup the user you will use for this lab and connect to a dedicated appliance allowing us to interact freely with the Data Fabric Environment

In [1]:
%login 16.31.86.198

echo "We are now connected to the appliance"

cat > $HOME/.bashrc << EOF
username="student1222" # your Data Fabric login credentials - username  
password="qcpx0k4u" # your Data Fabric login credentials - password - 
sec_username="sec-student1222" # your Data Fabric login credentials - sec username  
sec_password="stuDISCO2020" # your Data Fabric login credentials - sec password - 
apiserver_host="16.31.85.156" # API Server for the Data Fabric Cluster
ad_group="data-fabric-users" # LDAP Domain Group for Data Fabric users
vol_name="vol1" # suffix for student volume name
snapvol_name="snapvol1" # suffix for student snap volume name
cluster_name="mapr-cluster2.etc.fr.comm.hpecorp.net" # Data Fabric Cluster Name
src_vol_name="src-mirrvol1" # suffix for student source mirror volume name
dst_vol_name="dst-mirrvol1"  # suffix for student destination mirror volume name
secvol_name="sec-vol1"  # suffix for student volume name used for security comparison / user access 
sec_policy="student1222-secpol-$(uuidgen |cut -c 1-18)" #security policy to allow r/w access to volumes

EOF

echo "your operation context is:" student1222 "with Password:" qcpx0k4u

[ssh] Login to 16.31.86.198...
[ssh] host=16.31.86.198 hostname=16.31.86.198 other_conf={'user': None, 'port': None, 'keyfile': None, 'load_system_ssh_config': False, 'missing_host_policy': <paramiko.client.WarningPolicy object at 0x7f3b941b6700>}
[ssh] Successfully logged in.
[ssh] host = 16.31.86.198, cwd = /student/student1222
We are now connected to the appliance
your operation context is: student1222 with Password: qcpx0k4u


## 2. Let's start by creating a volume

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

#### The following command will create a new volume on the Data Fabric with:

In [2]:
echo "Volume Name: student1222-vol1"
echo "Mounted At: /user/student1222-vol1"
echo "The Accountable Entity Is Your User: student1222"

[ssh] host = 16.31.86.198, cwd = /student/student1222
Volume Name: student1222-vol1
Mounted At: /user/student1222-vol1
The Accountable Entity Is Your User: student1222


In [3]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$vol_name'&path=/user/'$username-$vol_name'' --user $username':'$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   174  100   174    0     0     70      0  0:00:02  0:00:02 --:--:--    70
{"timestamp":1624511404109,"timeofday":"2021-06-24 07:10:04.109 GMT+0200 AM","status":"OK","total":0,"data":[],"messages":["Successfully created volume: 'student1222-vol1'"]}

## 3. With the volume created we can now list out details on the volume


In [4]:
source ~/.bashrc

curl -k -X GET 'https://'$apiserver_host':8443/rest/volume/list?' --user $username:$password | jq -r | grep -B 6 -A 6 ''$username-$vol_name''

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  120k    0  120k    0     0   391k      0 --:--:-- --:--:-- --:--:--  392k
      "criticalReReplTimeOutSec": "0",
      "replicationtype": "high_throughput",
      "rackpath": "/data",
      "mirrorthrottle": "1",
      "accesstime": "June 24, 2021",
      "readonly": "0",
      "mountdir": "/user/student1222-vol1",
      "volumename": "student1222-vol1",
      "mounted": 1,
      "quota": "0",
      "advisoryquota": "0",
      "snapshotcount": 0,
      "logicalUsed": "0",
      "replicatedlogicalused": "0",
--
      "mirrorscheduleid": "0",
      "volumetype": "0",
      "mirrortype": 3,
      "creatorcontainerid": 0,
      "creatorvolumeuuid": "",
      "volumeid": 165343259,
      "actualreplication": "Information is not yet available for volume 'student1222-vol1'. Pleas

There are many volume attributes that can be set either on creation or after it is created by modifying the properties.  For a full listing of the attributes pleaess see:
[Volume Create Attributes](https://docs.datafabric.hpe.com/62/ReferenceGuide/volume-create.html)

Let's make a change to the volume you just created

## 4. Updating/change a property on volumes already created

In [5]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$vol_name'&securitypolicy=all-students' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0    401      0 --:--:-- --:--:-- --:--:--   402
{"timestamp":1624511419993,"timeofday":"2021-06-24 07:10:19.993 GMT+0200 AM","status":"OK","total":0,"data":[]}

In this example we just applied an already created security policy called 'all-students' to this volume. We will cover security policies in a future lab section. 

For a full listing of attributes you can change see the [Volume Modify documentation](https://docs.datafabric.hpe.com/62/ReferenceGuide/volume-modify.html)

## 5. Deleting volumes

Deleting volumes from a REST call is a very powerful command.  This can not be un-done, so use caution in a production environment.

In [6]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$vol_name'' --user $username:$password

[ssh] host = 16.31.86.198, cwd = /student/student1222
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   111  100   111    0     0     47      0  0:00:02  0:00:02 --:--:--    47
{"timestamp":1624511433539,"timeofday":"2021-06-24 07:10:33.539 GMT+0200 AM","status":"OK","total":0,"data":[]}

In [7]:
%logout 

[ssh] Closing existing connection.
[ssh] Successfully logged out.


## Creating, Updating, Deleting Summary

With a simple one line REST call you can manage your Ezmeral Data Fabric for all of your DevOps or DevSecOps needs for:

- Creating volumes
- Applying settings by modifying/editing volumes
- Deleting volumes 

## Next we are going to talk about protecting the data on the Ezmeral Data Fabric cluster.

* [Lab 2- Using Snapshots for Business Continuity](2-WKSHP-Snapshots.ipynb)
